In [89]:
import numpy as np
import pandas as pd

# Fungsi untuk menghitung limpasan berdasarkan metode CN
def calculate_limpasan(P, ARF, CN, Im):
    PSA007kum = [0.05, 0.15, 0.75, 0.91, 0.97, 1]
    PSA007 = np.diff(PSA007kum, prepend=0)
    Pjam = PSA007 * P
    Pjam_cum = np.cumsum(Pjam)
    # Tanpa Area Reduction F
    Pjam_ARF = Pjam * ARF  # Hujan Jam Jaman dikali dengan area reduction factor
    Pjam_ARF_cum = np.cumsum(Pjam_ARF)
    absis = np.arange(1, len(Pjam_ARF) + 1)  # Membuat array sumbu x (jumlah jam)

    P = np.array(Pjam_ARF)
    Pkum = np.array(Pjam_ARF_cum)

    def limpCN(CN, P):  # membuat fungsi dengan nilai CN dan Hujan
        '''P dalam mm, limp dalam mm'''
        CN_adj = CN+(100-CN)*(Im/100)
        S = 1000 / CN_adj - 10  # persamaan hubungan storage dengan nilai CN
        k = 0.2  # where K is varied between 0-0.26 (springer et al.), K=0.2 is recommended by SCS
        Pkum = []  # membuat array hujan kumulatif
        Ptot = 0  # mendefinisikan hujan total =0
        for i in range(len(P)):  # membuat nilai i sejumlah hujan jam-jaman yang diberikan
            Ptot += P[i] / 25.4  # Hujan total baru adalah hujan total sebelumnya di tambah hujan pada jam i (diubah dari mm ke inch)
            Pkum.append(Ptot)  # Menyimpan hujan kumulatif ke dalam array Pkum
        reff = []  # membua t array hujan efektif
        infil = []  # membuat array infiltrasi
        Iab = []  # membuat array Inital Abstraction
        reffkumseb = 0  # memberikan nilai hujan efektif sebelumnya
        Faseb = 0  # memberikan nilai infiltrasi sebelumnya
        for i in range(len(Pkum)):  # membuat nilai i sejumlah Hujan kumulatif
            Ia = k * S  # initial abtraction koefisien x dengan storage
            if (Pkum[i] > Ia):  # Cek apakah nilai hujan kumulatif pada jam tersebut lebih besar dari initial abtraction
                Ia = Ia  # jika hujan kumulatif > Ia maka berikan nilai Ia adalah Ia
                Iab.append(Ia)
            else:
                Ia = Pkum[i]  # jika hujan kumulatif < atau = Ia maka berikan nilai Ia adalah hujan kumulatif pada jam tersebut
                Iab.append(Ia)
            Fa = (S * (Pkum[i] - Ia) / (Pkum[i] - Ia + S)) # continuing abstraction (Fa) = storage x (hujan kumulatif - initial abstraction) / (hujan kumulatif - initial abstraction + storage)
            infil.append(Fa - Faseb)  # memasukan nilai infiltrasi yaitu continuing abstraction (Fa) - continuing abstraction sebelumnya (Fa)
            Faseb = Fa  # update nilai continuing abstraction (Fa) sebelumnya
            reffkum = Pkum[i] - Ia - Fa  # hujan efektif kumulatif = Hujan kumulatif - Ia -Fa
            reff.append(reffkum - reffkumseb)  # memasukan nilai hujan efektif = hujan efektif kumulatif - hujan efektif kumulatif sebelumnya
            reffkumseb = reffkum  # update nilai hujan kumulatif
        infil = np.array(infil) * 25.4  # membuat array infiltrasi dan mnejadikan dari inch ke mm
        reff = np.array(reff) * 25.4  # membuat array hujan dan menjadikan dari inch ke mm
        Iab = np.array(Iab) * 25.4 # membuat array intial abstraction dan menjadikan dari inch ke mm
        return (reff, infil, Iab)  # Menyimpan fungsi hujan efektif dan infiltrasi

    reff, infil, Iab = (limpCN(CN, P))  # Memanggil fungsi limpasan CN

    #Hujan Efektif Jam-jaman 
    
    infiltrasi_jam = (infil + Iab)
    reff_jam = reff

    infiltrasi_kum = np.cumsum(infiltrasi_jam)  
    reff_kum = np.cumsum(reff_jam)
    
    # Menyimpan hasil perhitungan dalam DataFrame
    refftab = {
        'Jam ke-': absis,
        'Hujan Rencana ': Pjam,
        'Hujan Rencana (ARF)': Pjam_ARF,
        'Infiltrasi': infiltrasi_jam,
        'Hujan Efektif': reff_jam,
        #'Nilai CN': CN,
        #'Nilai Impervious (%)': Im,
    }
    dfreff = pd.DataFrame(refftab)

    reffkumtab = {
        'Jam ke-': absis,
        'Hujan Rencana ': Pjam_cum,
        'Hujan Rencana (ARF)': Pkum,
        'Infiltrasi': infiltrasi_kum,
        'Hujan Efektif': reff_kum,
        #'Nilai CN': CN,s
        #'Nilai Impervious (%)': Im,
    }
    dfreffkum = pd.DataFrame(reffkumtab)  


    


In [92]:
P =100
ARF = 1
CN = 90
Im = 10

calculate_limpasan(P, ARF, CN, Im)

   Jam ke-  Hujan Rencana   Hujan Rencana (ARF)  Infiltrasi  Hujan Efektif
0        1             5.0                  5.0    5.000000       0.000000
1        2            10.0                 10.0   12.164490       2.835510
2        3            60.0                 60.0   16.368774      48.655401
3        4            16.0                 16.0    5.979879      15.044297
4        5             6.0                  6.0    5.315231       5.708945
5        6             3.0                  3.0    5.158798       2.865378
   Jam ke-  Hujan Rencana   Hujan Rencana (ARF)  Infiltrasi  Hujan Efektif
0        1             5.0                  5.0    5.000000       0.000000
1        2            15.0                 15.0   17.164490       2.835510
2        3            75.0                 75.0   33.533264      51.490912
3        4            91.0                 91.0   39.513143      66.535209
4        5            97.0                 97.0   44.828374      72.244154
5        6           100.